### 1- Create stats masks - Electrodes to be plotted
    Elec that reach the threshold in time (min sig)
    Create masks stats with 1000 perm

In [7]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product

conds = ['low','high']
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/3_Classif_PrefPha_E_EpiPerf_LowHigh_1000perm_theta_alpha/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = '{}_sources_{}_{}_odor_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_{}_minwin{}_th{}.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
fphases = ['2_theta', '3_alpha']# '0_VLFC', '1_delta', '2_theta', '3_alpha'
famps = ['0_beta','1_low-gamma','2_high-gamma']#,'1_low-gamma','2_high-gamma', '0_beta'
min_win = 1
th = 0.01

# # Find the permutation threshold by frequency band
# perm_max = np.array([])
# for famp,fpha in product(famps,fphases):
#     # Load the npz file with su_codes, xyz of signif and not signif elec
#     mat = np.load(f_form.format('All_subjects',famp,fpha))
#     s_perm = mat['s_perm']
#     #print(s_perm.shape)
#     perm_amp = np.array([])
#     for elec in range(s_perm.shape[0]):
#         perm = s_perm[elec]
#         th_perm = perm_pvalue2level(perm, p=th, maxst=True)
#         perm_amp = np.hstack((perm_amp,th_perm[0])) if np.size(perm_amp) else th_perm[0]
#     print(famp,fpha,max(perm_amp))
#     perm_max = np.hstack((perm_max,perm_amp)) if np.size(perm_max) else perm_amp
# thresh = max(perm_max) #find the max of perm thr across time and subjects
# print('perm max size',perm_max.shape,thresh)
thresh = 0.833333333333
# Create masks for a specific perm threshold
for famp,fpha in product(famps,fphases):
    mat = np.load(f_form.format('All_subjects',famp,fpha))
    s_da = mat['s_da']
    print(s_da.shape)
    mask = np.array([])
    for elec in range(s_da.shape[0]):
        da_elec = s_da[elec,:]
        pvals = np.array([])
        for win in range(da_elec.shape[0]):
            if da_elec[win] > thresh:
                pval = 1
            if da_elec[win] <= thresh:
                pval = 0
            pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
        #criteria for an elec to be significant
        pvals = np.ravel(pvals)
        underp = np.where(pvals == 1)[0]
        pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
        signif = [True for k in pvsplit if len(k) >= min_win]
        if len(signif) >=1:
            remove = False
        else:
            remove = True
        mask = np.hstack((mask, remove)) if np.size(mask) else remove
    print(famp,fpha, len(mask),len(np.where(mask == False)[0]))
    np.save(f_save.format('All_subjects',famp,fpha,min_win,th,'freqs'),mask)

-> Olfacto loaded
(456, 20)
0_beta 2_theta 456 0
(456, 20)
0_beta 3_alpha 456 8
(456, 20)
1_low-gamma 2_theta 456 2
(456, 20)
1_low-gamma 3_alpha 456 3
(456, 20)
2_high-gamma 2_theta 456 6
(456, 20)
2_high-gamma 3_alpha 456 1


### 2- Create masks - min patients

In [26]:
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
import numpy as np
from itertools import product
from PIL import Image

score = 'Epi'
conds = ['low','high']
th, corr = '0.05', False
###############################################################################
st = study('Olfacto')
path_classif = join(st.path, 'classified/2_Classif_PAC_Encoding_EpiPerf_LowHigh_1000perm/')
path_npz = join(st.path, 'figure/2_Classif_PAC_E_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'_1000perm/')
if corr == True:
    path_rois, path_stats = join(path_npz,'Signif_ROIs'+th+'/'), join(path_npz, 'masks_stat_corr/')
else :
    path_rois, path_stats = join(path_npz,'Signif_ROIs'+th+'/'), join(path_npz, 'masks_stat/')
path_to_save = join(path_npz, 'masks_visbrain'+th+'/')
path_save_fig = join(path_npz, 'Signif_timecourse'+th+'_corr/') if corr == True else join(path_npz, 'Signif_timecourse'+th+'/')
npz_form = join(path_npz, '{}_sources_{}_{}_odor_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz')
rois_form = join(path_rois, '{}_signif_min_{}_{}-{}_min_win{}_th_{}.npy')
stat_form = join(path_stats,'{}_mask_stat_{}_{}_minwin{}_th{}.npy')
mask_vis_form = join(path_to_save,'{}_mask_min{}_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
if not exists(path_save_fig):
    makedirs(path_save_fig)
############################################################################### 

fphases = ['0_VLFC', '1_delta', '2_theta', '3_alpha']# '0_VLFC', '1_delta', '2_theta', '3_alpha'
famps = ['0_beta','1_low-gamma','2_high-gamma']#,'1_low-gamma','2_high-gamma', '0_beta'
su_codes = {'S0':'CHAF','S1':'FERJ','S2':'LEFC','S3':'SEMC','S4':'PIRJ','S5':'MICP','S6':'VACJ'}
min_wins = ['1']#,'2','3','4','5']
methods = ['MAI_RL'] #,'aal_RL','BA','aal',
min_sigs = [1]#1,2,3,4,

for famp,fpha,min_win,min_sig,method in product(famps,fphases,min_wins,min_sigs,methods):
    # Load the electrodes labels, electrodes significant (bool), rois significant (names)
    npz_mat = np.load(npz_form.format('All_subjects',famp,fpha))
    print(npz_mat.files)
    labels0, labels, subjects = npz_mat['s_labels'],npz_mat['s_'+method],npz_mat['su_codes']
    elecs, channels = npz_mat['s_elec'],npz_mat['s_channels']
    stats = np.load(stat_form.format('All_subjects',famp,fpha,min_win,th))
    filename_rois = rois_form.format('s_Mai_RL',min_sig,famp[2:],fpha[2:],min_win,th)
    if exists(filename_rois):
        rois = np.load(filename_rois)
        mask_vis = []
        i=0
        for e in range(labels.shape[0]):
            if stats[e]==0 and labels[e] in rois:
                mask_vis.append(False)
                i=i+1
                print(subjects[e],labels[e],channels[e],elecs[e],'number of elecs',i)
                #path_file = path_classif+su_codes[subjects[e]]+'/'+freq+'/fig/'
                path_file_time = path_classif+famp+'/'+fpha+'/fig/'
                filename = su_codes[subjects[e]]+'_Pac_'+conds[0]+'_'+conds[1]+'_'+channels[e]+'_'+labels[e]+'_('+elecs[e]+').png'
                img = Image.open(path_file_time+filename)
                path_save_fig2 = path_save_fig+famp+'/'+fpha+'/'+method+'/'+labels[e]+'/'
                if not exists(path_save_fig2):
                    makedirs(path_save_fig2)
                img.save(path_save_fig2+'/min_patient'+str(min_sig)+'_minwin_'+str(min_win)+'_'+filename,'png')
            else :
                mask_vis.append(True)
            #print(stats[e],labels[e], mask_vis[e])
        np.save(mask_vis_form.format(method,min_sig,famp,fpha,min_win,th),mask_vis)

-> Olfacto loaded
['s_perm', 's_elec_feat0', 's_da', 's_elec_feat1', 's_xyz', 's_elec', 's_labels', 's_aal', 'su_codes', 's_MAI', 's_BA', 's_aal_RL', 's_channels', 's_MAI_RL']
['s_perm', 's_elec_feat0', 's_da', 's_elec_feat1', 's_xyz', 's_elec', 's_labels', 's_aal', 'su_codes', 's_MAI', 's_BA', 's_aal_RL', 's_channels', 's_MAI_RL']
S6 IFG e'5-e'4 25 number of elecs 1
['s_perm', 's_elec_feat0', 's_da', 's_elec_feat1', 's_xyz', 's_elec', 's_labels', 's_aal', 'su_codes', 's_MAI', 's_BA', 's_aal_RL', 's_channels', 's_MAI_RL']
S1 IFG o8-o7 75 number of elecs 1
S4 STG t5-t4 46 number of elecs 2
S6 MTG b12-b11 7 number of elecs 3
S6 ACC k'2-k'1 39 number of elecs 4
S6 IFG k'13-k'12 41 number of elecs 5
S6 ITG l9-l8 50 number of elecs 6
S6 OFC o'3-o'2 64 number of elecs 7
S6 OFC o'10-o'9 69 number of elecs 8
S6 STG t8-t7 71 number of elecs 9
['s_perm', 's_elec_feat0', 's_da', 's_elec_feat1', 's_xyz', 's_elec', 's_labels', 's_aal', 'su_codes', 's_MAI', 's_BA', 's_aal_RL', 's_channels', 's_MAI_R